In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [2]:
''' Parameters used for all networks (reciprocal and chain)'''
N =1500#2000
J = 1/np.sqrt(1500)*0.5
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

c = 0.2
# sigma2 = N*c*(1-c)*J**2
g, gamma = 6.0, 1/4.0#1.5,1/1.# 6.8, 1/4.0#1.5,1/1.#
NE = int(N/(1+gamma))
NI = int(gamma*NE)
N  = NE+NI
print(N,NE,NI)
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(c*NE), int(c*NI) ### fixed out-degree
print('number of connected E/I neurons:',KE,KI)
print('non-zero J:',J)
ji,je = g*J,J 

### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
print('lambda0:',lambda0)
print('ge,gi:',ge,gi)
print('JE,JI:',JE,JI)

### construct the mean matrix


e:\Dropbox\DailyWork\Allen_project\Preparation_Paper_Figures\PRX_prune\PRX_numeric
1500 1200 300
number of connected E/I neurons: 240 60
non-zero J: 0.012909944487358056
lambda0: -1.5491933384829668
ge,gi: 0.2 1.2000000000000002
JE,JI: 3.0983866769659336 4.6475800154489


In [8]:
# for value in {0.   0.14 0.28 0.42 0.56 0.7  0.84 0.98 1.12 1.26 1.4 }; do
#     ./run_secorder 1000 0.2 $(echo "$value*2" | bc) $value $value 1 1
# done
# for seed in {1 7 8 12 13 19 28 31 33 39 40 42 48 50 51 52 55 56 59 65 66 67 69 70 75 76 78 82 83 86 94}; do     mkdir -p "seed_$seed"; done

In [3]:
ntau   = 10#21#
trials = 30+6
tau_series = np.linspace(0,0.225,ntau)# np.linspace(0.25,0.4,ntau)#
alpha_series = (tau_series*c*(1-c)+c**2)/c**2-1
alpha_series_rec = (tau_series*2*c*(1-c)+c**2)/c**2-1
print(alpha_series)
print(alpha_series_rec)

((alpha_series+1)*c**2-c**2)/c/(1-c)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8]


array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225])

In [4]:
# tau_series = np.array([0.025, 0.05 , 0.075, 0.1  ])
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### normalization for left and right eigenvectors 
norms_for_lvec, norms_for_rvec = np.zeros((trials,ntau+1,2)), np.zeros((trials,ntau+1,2))
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]  = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left

eigvJ0: (-1.5491933384830117+0j)  theory: -1.5491933384829668


### Chain motifs

In [5]:
alphachn_series = np.array([0.000,0.100,0.200,0.300,0.400,0.500,0.600,0.700,0.800,0.900])

In [6]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
shiftx = 1.5
def odeIntegralP(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@(1.0+np.tanh(x-shiftx))
	return np.squeeze(dxdt)
def odesimulationP(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegralP,J=Jpt,I=I),xinit,t)

In [7]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [8]:
ce=ci=c
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.01002712 0.01009151]


In [9]:
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population


In [56]:
trials = 36
alphachns     = ['0.000','0.100','0.200','0.300','0.400','0.500','0.600','0.700','0.800','0.900']
# alphachns_arr = np.array([0.1,0.2,0.3,0.4])
eigvchn_series = np.zeros((trials,len(alphachns),N),dtype=complex)
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))
norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))

intg_mean_series  = np.zeros((trials,ntau,N,2,3))### mean overlap matrix rank and 
# tauchns_arr = alphachns_arr*c**2/c/(1-c)
seedss = np.array([1, 4, 8, 10, 13, 14, 18, 19, 25, 26, 31, 32, 34, 36, 40, 43, 44, 49, 52, 53, 55 ,60, 62, 64, 71, 73, 76, 78, 81, 82, 83,84, 85, 86, 87, 88, 93, 94, 95])
seedss = np.arange(1,37)
for ktrial in range(trials):
    seed =seedss[ktrial]
    z_pre = 0
    for idxalpha, alphachn in enumerate(alphachns[:]):
        # if idxalpha <8:
        #     continue
        file_path = 'J:\\Allen_motif\\PRXL_data\\sonets\\N1500_reciprocal\\seed_'+str(seed)+'\\w_1500_0.200_'+alphachns[idxalpha]+'_0.000_0.000_0.000_'+str(seed)+'.dat'
        Jmat = np.zeros((N,N))
        count =0
        with open(file_path, 'r') as file:
            content = file.read()
            if content == '0':
                print('File is empty') 
            else:
                for line in content.split('\n'):
                    if line:
                        Jmat[count,:]=([(x) for x in line.split()])
                        count +=1          
        Jmat[:,:NE] = Jmat[:,:NE]*je
        Jmat[:,NE:] = Jmat[:,NE:]*(-ji)

        ktrial, it = ktrial, idxalpha
        ### full rank simulation
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, Jmat, Inp)
        firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
            
        zr = Jmat - Jbar
        ### compute the eigenvalues and eigenvectors of the matrix
        eigvchn, eigrvec = la.eig(Jmat)
        ## sort the eigenValues by it's norm of the real part 
        idxeigv = np.argsort(np.abs(eigvchn))
        idxeigv = idxeigv[::-1]
        eigvchn, eigrvec = eigvchn[idxeigv],eigrvec[:,idxeigv]
        
        eigvchn_,eiglvec = la.eig(Jmat.copy().T)
        idxeigv = np.argsort(np.abs(eigvchn_))
        idxeigv = idxeigv[::-1]
        eigvchn_, eiglvec = eigvchn[idxeigv],eiglvec[:,idxeigv]
        
        ### normalization
        reig    = np.squeeze(eigrvec[:,:].copy())
        leig0   = np.squeeze(eiglvec[:,:].copy()) 
        normval = np.sum(reig.copy()*leig0.copy(),axis=0)
        
        norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
        normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
        if np.mean(reig[:NE,0])<0:
            reig[:,0]*=-1
            leig[:,0]*=-1
            leig0[:,0]*=-1
        if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
            reig[:,1]*=-1
            leig[:,1]*=-1
            leig0[:,1]*=-1
            
        ### numerical low-rank approximation 
        ov_inp_lowrank,ov_inp_lowrank_div = np.zeros(2),np.zeros(2)
        vec_lowrank_contribution = np.zeros((N,2))
        ### linear response theory approximation 
        for i in range(2):
            ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
            ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
        Equilibrium_lowrank_outliers = np.reshape(Inp.copy(),(N,1))
        # print('EQ shape:',np.shape(Equilibrium_lowrank_outliers))
        for i in range(2):
            vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
            # print('shape :...',np.shape(np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))))
            Equilibrium_lowrank_outliers += np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))
            #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
            contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
            contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
        lowrank_eq_num[ktrial,it,:] = np.squeeze(Equilibrium_lowrank_outliers.copy())
        ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 
          
        print('seed:',seed,'alphachn:',alphachn,'idxalpha:',idxalpha)
        print('eigenvalues',eigvchn[:2])
        
        DELTA_Z = np.zeros((N,N))
        DELTA_Z = zr-z_pre  
        hzr = DELTA_Z.copy()
        
        eigvchn_series[ktrial,it,:] = eigvchn[:] 
        eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
        eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
        eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
        for iii in range(2):
            eiglvec0norm_series[ktrial,it,:,iii] = leig0[:,iii].copy()/normval[iii,iii]
            
        DeltaZ2 = hzr@hzr    ### used to correct        
        if it<1:
            lvec, rvec = np.squeeze(leigvec0[:,:2]),np.squeeze(reigvec0[:,:2])
            # eigeng     = np.squeeze(eigvchn_series[ktrial,it,:2].copy())
            eigeng     = np.array([JE-JI,0])
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
        else:
            lvec, rvec = np.squeeze(eiglvec0_series[ktrial,it-1,:,:2]),np.squeeze(eigrvec_series[ktrial,it-1,:,:2]) ### use the previous eigenvector as the initial condition   
            eigeng = np.squeeze(eigvchn_series[ktrial,it-1,:2].copy())
            ### conditioned mean  
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
            lvec_mean[:NE,:],lvec_mean[NE:,:]=np.mean(lvec_mean[:NE,:],axis=0),np.mean(lvec_mean[NE:,:],axis=0)
            rvec_mean[:NE,:],rvec_mean[NE:,:]=np.mean(rvec_mean[:NE,:],axis=0),np.mean(rvec_mean[NE:,:],axis=0)
            
        ### get the appropriate normalization factor
        norm_for_lvec, norm_for_rvec = np.zeros(2),np.zeros(2)
        ### for the first delta tau, there is no change in the structure of the eigenvectors (only the random part changes due to the iid random connectivity)
        if it==0:
            hzr_u = zr.copy()
            with_chn = 0
        else:
            hzr_u = hzr.copy()
            with_chn = 1
                    
        ### predicted eigenvectors based on the previous eigenvectors(not the real one)
        ### record the reconstructed eigenvectors
        intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
        norm_rvec_temp, norm_lvec_temp = np.zeros((N,2)), np.zeros((N,2))
        for i in range(2):
            rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
            lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
    
            ''' Mean connectivity '''                
            current_eigv = eigvchn_series[ktrial,it,i].copy()
            if it==0:
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                ''' otherwise no correlation can be calculated '''
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            elif it < 5 and i==1:#it < 3 and i==1:### 7 is 2 blocks after radius<outlier
                ### original values are obtained numerically
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                ''' otherwise no correlation can be calculated '''
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            else:
                eigenvalue_u = np.real(current_eigv)
                eigenvalue_um = np.real(eigeng[i]) 
                
                ### norm_for_rvec and norm_for_lvec 
                rmean_tmp =np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*(DeltaZ2@np.reshape(rvec_mean[:,i],(-1,1)))/np.real(eigenvalue_um)**2,(N,1))### mean-shifting 
                rvec_n[:NE,0] = rvec_n[:NE,0] - np.mean(rvec_n[:NE,0])+rmean_tmp[:NE,0]
                rvec_n[NE:,0] = rvec_n[NE:,0] - np.mean(rvec_n[NE:,0])+rmean_tmp[NE:,0]
                
                ### raw, unnormalized
                lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/np.real(eigeng[i])**2,(N,1))### mean-shifting 
                lvec_n[:NE,0] = lvec_n[:NE,0] - np.mean(lvec_n[:NE,0])+lmean_tmp[:NE,0]
                lvec_n[NE:,0] = lvec_n[NE:,0] - np.mean(lvec_n[NE:,0])+lmean_tmp[NE:,0]
                
                norm_for_rvec[i] = la.norm(rvec_n) ### normalization factor 
                norm_rvec_n = np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i] 
                norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
                
                norm_for_lvec[i] = np.squeeze(np.reshape(lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))     
                norm_lvec_n = np.reshape(lvec_n.copy(),(-1,1))/norm_for_lvec[i] 
                ### need to be re-normalized
                norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())
                norm_lvec_temp[:,i] *= eigvchn_series[ktrial,it,i].real
                
                ### more simplified version using lvec_mean and rvec_mean
                intg_ml[:,i] = lmean_tmp[:,0]/norm_for_lvec[i]*eigenvalue_u
                leig0mean_series[ktrial,it,:,i] =lmean_tmp[:,0]/norm_for_lvec[i]
                leig0pre_series[ktrial,it,:,i] =np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/np.real(eigeng[i])**2,(N,1))[:,0]#*eigenvalue_u#
                intg_mr[:,i] = rmean_tmp[:,0]/norm_for_rvec[i]
                
                tilden = np.reshape(lvec_n.copy(),(-1,1))/norml0_series[ktrial,it,i]
                norm_4lvec_series_[ktrial,it,i]=np.reshape(tilden,(1,-1))@np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i]
                
                norm_4rvec_series[ktrial,it,i],norm_4lvec_series[ktrial,it,i]=norm_for_rvec[i],norm_for_lvec[i]
                if i==0:
                    print(i,'should be the same',norm_4lvec_series[ktrial,it,i]/norml0_series[ktrial,it,i],norm_4lvec_series_[ktrial,it,i])
                
            
            ### @YS 17 Nov, no matter what the variable it is.
            ### modify the mean of the elements on the left and right eigenvectors
            norm_rvec_temp[:NE,i] -= np.mean(norm_rvec_temp[:NE,i])
            norm_rvec_temp[NE:,i] -= np.mean(norm_rvec_temp[NE:,i])
            norm_rvec_temp[:NE,i] += np.mean(intg_mr[:NE,i])
            norm_rvec_temp[NE:,i] += np.mean(intg_mr[NE:,i])
            norm_lvec_temp[:NE,i] -= np.mean(norm_lvec_temp[:NE,i])
            norm_lvec_temp[NE:,i] -= np.mean(norm_lvec_temp[NE:,i])
            norm_lvec_temp[:NE,i] += np.mean(intg_ml[:NE,i])
            norm_lvec_temp[NE:,i] += np.mean(intg_ml[NE:,i])
                
            ### record the reconstructed eigenvectors
            eigrvec_series_rec[ktrial,it,:,i] = norm_rvec_temp[:,i].copy()
            eiglvec_series_rec[ktrial,it,:,i] = norm_lvec_temp[:,i].copy()
            ### record the mean of the elements on the left and right eigenvectors  
            intg_mean_series[ktrial,it,:,i,0] = np.reshape(intg_ml[:,i],(N,))
            intg_mean_series[ktrial,it,:,i,1] = np.reshape(intg_mr[:,i],(N,))
            
        '''overlap per population '''
        ### theoretically compute the equilibrium population-averaged firing rate
        an = np.zeros((2,2),dtype=complex)
        am = np.zeros((2,2),dtype=complex) ## population X rank    
        for ir in range(2):
            an[0,ir] = np.mean(intg_ml[:NE,ir])
            an[1,ir] = np.mean(intg_ml[NE:,ir])
            am[0,ir] = np.mean(intg_mr[:NE,ir])
            am[1,ir] = np.mean(intg_mr[NE:,ir])
            
        ### overlap sum
        overlap_inp = np.zeros(2,dtype=complex) 
        for ir in range(2):
            overlap_inp[ir] = (NE*an[0,ir]*Inp[0]+NI*an[1,ir]*Inp[-1])
            ovs_inplr[ktrial,it,ir] = overlap_inp[ir] ### recording_theory
            overlap_inp[ir]/= (1.0-eigvchn[ir])
            ovs_inplr_div[ktrial,it,ir] = overlap_inp[ir] ### recording theory
        eq_fr = np.zeros(2)
        eq_fr[0],eq_fr[1] = Inp[0],Inp[-1]
        for ir in range(2): ### rank two 
            ### record 
            contributions_lr[ktrial,it,ir,0] = overlap_inp[ir]*am[0,ir] ### excitatory population
            contributions_lr[ktrial,it,ir,1] = overlap_inp[ir]*am[1,ir] ### inhibitory population
            eq_fr[0] += am[0,ir]*overlap_inp[ir]
            eq_fr[1] += am[1,ir]*overlap_inp[ir]
        lowrank_eq[ktrial,it,:] = eq_fr.copy() 
    
        
        z_pre = zr.copy()    

C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:60: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:77: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:82: ComplexWarning: Casting complex values to real discards the imaginary part
  vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:99: ComplexWarning: Casting complex values to re

seed: 1 alphachn: 0.000 idxalpha: 0
eigenvalues [-1.48238059+0.j          0.48153101+0.34732327j]
seed: 1 alphachn: 0.100 idxalpha: 1
eigenvalues [-1.42987878+0.j          0.51866913+0.26790187j]
0 should be the same 1.009975110770241 1.0099751107702415


C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:148: ComplexWarning: Casting complex values to real discards the imaginary part
  intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:149: ComplexWarning: Casting complex values to real discards the imaginary part
  intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:150: ComplexWarning: Casting complex values to real discards the imaginary part
  leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:152: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:153: ComplexWa

seed: 1 alphachn: 0.200 idxalpha: 2
eigenvalues [-1.45868687+0.j          0.4996107 +0.31322399j]
0 should be the same 1.022847038366001 1.0228470383660009
seed: 1 alphachn: 0.300 idxalpha: 3
eigenvalues [-1.45510751+0.j          0.5271815 +0.27548011j]
0 should be the same 1.0065968169011303 1.0065968169011317
seed: 1 alphachn: 0.400 idxalpha: 4
eigenvalues [-1.45995242+0.j          0.55654686+0.26709952j]
0 should be the same 1.0039081296942551 1.0039081296942542
seed: 1 alphachn: 0.500 idxalpha: 5
eigenvalues [-1.4406856 +0.j          0.54468216+0.28290271j]
0 should be the same 1.0081517400546125 1.0081517400546118
seed: 1 alphachn: 0.600 idxalpha: 6
eigenvalues [-1.45651717+0.j -0.60407514+0.j]
0 should be the same 1.0117210810484305 1.0117210810484314
seed: 1 alphachn: 0.700 idxalpha: 7
eigenvalues [-1.47862082+0.j -0.62119901+0.j]
0 should be the same 1.0067284278092612 1.006728427809261
seed: 1 alphachn: 0.800 idxalpha: 8
eigenvalues [-1.47325937+0.j -0.62577722+0.j]
0 should b

C:\Users\Administrator\AppData\Local\Temp\ipykernel_166216\3330372010.py:134: RuntimeWarning: invalid value encountered in divide
  rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])


seed: 2 alphachn: 0.100 idxalpha: 1
eigenvalues [-1.52925734+0.j  0.62781276+0.j]
0 should be the same 1.016671080826365 1.016671080826365
seed: 2 alphachn: 0.200 idxalpha: 2
eigenvalues [-1.52927445+0.j  0.62696074+0.j]
0 should be the same 0.9966213581042815 0.9966213581042795
seed: 2 alphachn: 0.300 idxalpha: 3
eigenvalues [-1.53489229+0.j          0.59935964+0.01077649j]
0 should be the same 1.0078755585272814 1.007875558527281
seed: 2 alphachn: 0.400 idxalpha: 4
eigenvalues [-1.53354049+0.j  0.60109295+0.j]
0 should be the same 1.0039083332238876 1.003908333223887
seed: 2 alphachn: 0.500 idxalpha: 5
eigenvalues [-1.57038403+0.j  0.61453373+0.j]
0 should be the same 1.0149456110850308 1.0149456110850306
seed: 2 alphachn: 0.600 idxalpha: 6
eigenvalues [-1.57178504+0.j         0.57015388+0.1896817j]
0 should be the same 0.9988411276891009 0.9988411276891004
seed: 2 alphachn: 0.700 idxalpha: 7
eigenvalues [-1.59234443+0.j          0.58415307+0.18695627j]
0 should be the same 0.9962297

In [ ]:

# for seed in {0..36}; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed

# done


# for seed in {1..37}; do
# ./run_secorder 2000 0.15 1.2 0.6 0.6 1 $seed
# done


# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed
# done

# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do     mkdir -p "seed_$seed"; done

In [23]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [10]:
# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'c':c,
#           'J':J,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         lowrank_eq, firing_rateeq,lowrank_eq_num,
#         ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
#         contributions_lr, contributions_lr_num,
#         intg_mean_series,leig0mean_series,
#         norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "lowrank_eq, firing_rateeq,lowrank_eq_num,"
#         "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
#         "contributions_lr, contributions_lr_num,"
#         "intg_mean_series,leig0mean_series,"
#         "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
# data = list_to_dict(lst=lst, string=stg)
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/DynsStats_WD_Adjacency_4April_PRX_60N1500_normlr_reciprocal.npz"
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/DynsStats_WD_Adjacency_4April_PRX_60N1500_normlr_reciprocal.npz"
# np.savez(data_name, **data)
# ### load the stored data, in particular, extracting variable: eigvchn_series 
# data = np.load(data_name,allow_pickle=True)

# ### also loading other variables 
# eigvchn_series = data['eigvchn_series']
# eigrvec_series = data['eigrvec_series']
# eiglvec_series = data['eiglvec_series']
# eigrvec_series_rec = data['eigrvec_series_rec']
# eiglvec_series_rec = data['eiglvec_series_rec']
# eiglvec0_series = data['eiglvec0_series']
# norml0_series = data['norml0_series']
# params = data['params']
# intg_mean_series = data['intg_mean_series']
# leig0mean_series = data['leig0mean_series']


### load data
data = np.load(data_name,allow_pickle=True)
firing_rateeq = data['firing_rateeq']
lowrank_eq    = data['lowrank_eq']
lowrank_eq_num = data['lowrank_eq_num'] 
contributions_lr = data['contributions_lr'] ### trial, tau, rank, pop
contributions_lr_num = data['contributions_lr_num']
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_mean_series = data['intg_mean_series']
leig0mean_series = data['leig0mean_series']


In [11]:
### compute the theoretical outliers and the spectral radius
### recording
eigvchn_theo = np.zeros((ntau,2))
radius_theo_map = np.zeros(ntau)
### print variables and parameters of the adjacency matrix 
print('-------------------')
print('N:',N)
print('g:',g)
print('gamma:',gamma)
print('J:',J)
print('JE,JI:',JE,JI)
print('ge,gi:',ge,gi)
print('c:',c)
print('je,ji:',je,ji)
print('-------------------')


-------------------
N: 1500
g: 6.0
gamma: 0.25
J: 0.012909944487358056
JE,JI: 3.0983866769659336 4.6475800154489
ge,gi: 0.2 1.2000000000000002
c: 0.2
je,ji: 0.012909944487358056 0.07745966692414834
-------------------


In [12]:
ce, ci = c,c
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([0,0])
    tau_con_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[tau,tau],[tau,tau]])
    tau_chn_vec = np.array([[0,0],[0,0]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo

In [13]:
Z2E = N*J**2*hat_sigmae**2*tau_series*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI ### only diagonal terms
Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI ### only diagonal terms
DeltaZ2E, DeltaZ2I = Z2E[1]-Z2E[0],Z2I[1]-Z2I[0] ### for practical use
#### solve the eigenvalues using 4th order polynomial equation
eigvchn_theo_4th = np.zeros((ntau,4))
for it in range(ntau):
    a = 1.0
    b = -(JE-JI)
    cc = -(Z2E[it]+Z2I[it])
    d = (JE*Z2I[it]-JI*Z2E[it])
    e = Z2E[it]*Z2I[it]
    
    ## solve the polynomial
    xinit = np.mean(eigvchn_series[:,it,0].real)
    x_pred = np.roots([a,b,cc,d,e])
    # print('the first prediction:',x_pred)
    eigvchn_theo_4th[it,0] = x_pred[0]
    xinit = np.mean(eigvchn_series[:,it,1].real)
    x_pred = np.roots([a,b,cc,d,e])
    # print('the second prediction:',x_pred)
    eigvchn_theo_4th[it,1] = x_pred[0] 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8164\4218715285.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  eigvchn_theo_4th[it,0] = x_pred[0]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8164\4218715285.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  eigvchn_theo_4th[it,1] = x_pred[0]


In [14]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [15]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [32 11  6  7 23 21 26  1 14  3 10 20 22 13 34  9 12 28 15 18  0  5  8 30
  2 33 17 19  4 31] 30


In [75]:
### plot the numerical and theoretical eigenvalues outliers
# fig,ax = plt.subplots(figsize=(6,4))
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,eigvchn_theo_4th[:,0],'tab:red',label='theoretical')
ax.plot(tau_series,(JE-JI)*np.ones_like(tau_series),'k',ls='--')

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-2.5,0.8])
ax.set_yticks([-2.5,0,0.5])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [84]:
### plot y==1 
# ax.axhline(y=1, color='k', linestyle='-')

In [21]:
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
norm_4lvec_series_select = norm_4lvec_series.copy()
norm_4lvec_series_select_ = norm_4lvec_series_.copy()
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(norm_4lvec_series[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select[idxnan,it,ir] = np.nan 
        
        idxsort = np.argsort(norm_4lvec_series_[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select_[idxnan,it,ir] = np.nan 

In [22]:
#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:]-0.005,np.nanmean(norm_4lvec_series_select[:,1:,0]/norml0_series[:,1:-1,0],axis=0),yerr=np.nanstd(norm_4lvec_series_select[:,1:,0]/norml0_series[:,1:-1,0],axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:]+0.005,np.mean(norm_4rvec_series[:,1:,0],axis=0),yerr=np.std(norm_4rvec_series[:,1:,0],axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:],np.nanmean(norm_4lvec_series_select_[:,1:,0].real,axis=0),yerr=np.nanstd(norm_4lvec_series_select_[:,1:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:],np.mean(norm_4rvec_series[:,1:,0].real,axis=0),yerr=np.std(norm_4rvec_series[:,1:,0].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

In [29]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_reciprocal.npz"
data = np.load(data_name,allow_pickle=True)

### also loading other variables 
norm_4lvec_series = data['norm_4lvec_series']
norm_4rvec_series = data['norm_4rvec_series']
norm_4lvec_series_ = data['norm_4lvec_series_']

In [30]:
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
norm_4lvec_series_select = norm_4lvec_series.copy()
norm_4lvec_series_select_ = norm_4lvec_series_.copy()
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(norm_4lvec_series[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select[idxnan,it,ir] = np.nan 
        
        idxsort = np.argsort(norm_4lvec_series_[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select_[idxnan,it,ir] = np.nan 

In [31]:
#### plot the norm_4lvec and norm_4rvec change with tau^r   
# #### plot the norm_4lvec and norm_4rvec change with tau^r   
# fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
cc ='black'#'gray'#
ax.errorbar(tau_series[1:],np.nanmean(norm_4lvec_series_select_[:,1:,0].real,axis=0),yerr=np.nanstd(norm_4lvec_series_select_[:,1:,0].real,axis=0),fmt='x',color=cc,ecolor=cc,alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])
# ax.plot([-0.1,0.3],[1,1],color='gray',linestyle='--',linewidth=1.5,alpha=0.5)

#### Dynamics

In [16]:
### compute the population mean and variance
firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)
firing_rateeq_var[:,:,0],firing_rateeq_var[:,:,1] = np.var(firing_rateeq[:,:,:NE],axis=2),np.var(firing_rateeq[:,:,NE:],axis=2)

lowrank_eq_num_mean,lowrank_eq_num_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
lowrank_eq_num_mean[:,:,0],lowrank_eq_num_mean[:,:,1] = np.mean(lowrank_eq_num[:,:,:NE],axis=2),np.mean(lowrank_eq_num[:,:,NE:],axis=2)
lowrank_eq_num_var[:,:,0],lowrank_eq_num_var[:,:,1] = np.var(lowrank_eq_num[:,:,:NE],axis=2),np.var(lowrank_eq_num[:,:,NE:],axis=2)

In [17]:
frate_real = firing_rateeq_mean[:,-1,0].copy()
### sorting eigvchn_real 
idx = np.argsort(frate_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 2  4  8 33 27 15 24  9 18 26  5  7 31 21 10  0 12 16 28  3 23 20 34 13
 30 14  1 35 25 11] 30


In [18]:
#### filter the firing rate
epsilon = 1e-3
thresh_low = 2.0/10.0
### select trials
ineffective = []
for ktrial in range(trials):
    if eigvchn_series[ktrial,-1,0].real>eigvchn_series[ktrial,-1,1].real:
        print('in...',ktrial)
        ineffective = np.append(ineffective,ktrial)
        continue 
    if np.abs(eigvchn_series[ktrial,-1,0].imag)>epsilon:
        ineffective = np.append(ineffective,ktrial)
        continue
    if np.abs(eigvchn_series[ktrial,-1,1].imag)>epsilon:
        ineffective = np.append(ineffective,ktrial)
        continue
effective_trials = np.setdiff1d(np.arange(trials),ineffective)   

cut = 3
for irr in range(2):
    for it in range(ntau):
        idxsort = np.argsort(lowrank_eq[:,it,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq[idxnan,it,0] = np.nan 
        idxsort = np.argsort(lowrank_eq[:,it,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq[idxnan,it,1] = np.nan 
        ### same for lowrank_eq_num_mean 
        idxsort = np.argsort(lowrank_eq_num_mean[:,it,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq_num_mean[idxnan,it,0] = np.nan
        idxsort = np.argsort(lowrank_eq_num_mean[:,it,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq_num_mean[idxnan,it,1] = np.nan
        
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau): 
            if ktrial in ineffective:
                lowrank_eq[ktrial,it,0] = np.nan
                lowrank_eq[ktrial,it,1] = np.nan
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
                continue
            if np.abs(eigvchn_series[ktrial,it,irr].imag)>epsilon:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
                lowrank_eq[ktrial,it,0] = np.nan
                lowrank_eq[ktrial,it,1] = np.nan
                continue
            
            ### keep the better match solutions
            if np.abs(lowrank_eq_num_mean[ktrial,it,0])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq_num_mean[ktrial,it,1])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
            if np.abs(lowrank_eq[ktrial,it,0])>thresh_low:
                lowrank_eq[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq[ktrial,it,1])>thresh_low:
                lowrank_eq[ktrial,it,1] = np.nan
                
for it in range(ntau):
    idxsort = np.argsort(firing_rateeq_mean[:,it,0])
    idxnan = np.append(idxsort[:cut],idxsort[-cut:])
    firing_rateeq_mean[idxnan,it,0] = np.nan
    
    idxsort = np.argsort(firing_rateeq_mean[:,it,1])
    idxnan = np.append(idxsort[:cut],idxsort[-cut:])
    firing_rateeq_mean[idxnan,it,1] = np.nan

for ktrial in range(trials):
    for it in range(0,ntau):                
        if np.abs(firing_rateeq_mean[ktrial,it,0])>thresh_low:
            firing_rateeq_mean[ktrial,it,0] = np.nan
        if np.abs(firing_rateeq_mean[ktrial,it,1])>thresh_low:
            firing_rateeq_mean[ktrial,it,1] = np.nan
            
for irr in range(2):
    for it in range(ntau):
        idxsort = np.argsort(contributions_lr[:,it,irr,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr[idxnan,it,irr,0] = np.nan
        
        idxsort = np.argsort(contributions_lr[:,it,irr,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr[idxnan,it,irr,1] = np.nan
        
        idxsort = np.argsort(contributions_lr_num[:,it,irr,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr_num[idxnan,it,irr,1] = np.nan
        
        idxsort = np.argsort(contributions_lr_num[:,it,irr,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr_num[idxnan,it,irr,0] = np.nan
                
#### same for the contribution from different terms
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau):
            if ktrial in ineffective:
                contributions_lr[ktrial,it,irr,0] = np.nan
                contributions_lr[ktrial,it,irr,1] = np.nan
                contributions_lr_num[ktrial,it,irr,0] = np.nan
                contributions_lr_num[ktrial,it,irr,1] = np.nan
                continue
            ### keep the better match solutions
            if np.abs(contributions_lr[ktrial,it,irr,0])>thresh_low:
                contributions_lr[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr[ktrial,it,irr,1])>thresh_low:
                contributions_lr[ktrial,it,irr,1] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,0])>thresh_low:
                contributions_lr_num[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,1])>thresh_low:
                contributions_lr_num[ktrial,it,irr,1] = np.nan
                


In [19]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']

mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
rankk = 1 ### reciprocal motifs only rank-1
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(rankk):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
                
            mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(rankk):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])        
            
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6#53,54#3#55#
for it in range(ntau):
    for ir in range(rankk):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        if it>=6 and ir==1:
            idxpos=np.where(mean_leigvec_series[:,it,ir,0].real>0)[0]
            mean_leigvec_series[idxpos,it,ir,0] = np.nan
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        if it>=6 and ir==1:
            idxneg=np.where(mean_leigvec_series[:,it,ir,1].real<0)[0]
            mean_leigvec_series[idxneg,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,0].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,1].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        
        

In [71]:
dtau = tau_series[1]-tau_series[0]
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.01,0.08)
ax[0].set_yticks([0,0.05])
# ax[0].set_ylim(-1,8)
# ax[0].set_yticks([-1,8])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)

fig.tight_layout()


In [72]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,0],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$E$')
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[idx_eff,:,0,0]*eigvchn_series[idx_eff,:,0],axis=0),color='black',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### excitatory population
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.2,0.35)
ax[0].set_yticks([0,0.3])
# ax[0].set_ylim(-12,35)
# ax[0].set_yticks([-10,35])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,1],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),color='orange',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-1.5,1)
ax[1].set_yticks([-1,0])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5398: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[0] = start
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5399: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N + 1] = end
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5402: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5404: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N+2:, 1] = dep2slice[::-1]
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [20]:
### compute low-rank theoretical
contribution_theo = np.zeros((ntau,rankk,2))
for it in range(ntau):
    if it<2:
        for ir in range(1):
            contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/(1-eigvchn_theo_4th[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))
            contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/(1-eigvchn_theo_4th[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))
    else:
        for ir in range(1):
            contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/(1-eigvchn_theo_4th[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))
            contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/(1-eigvchn_theo_4th[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))
firing_rate_lwtheo = np.ones((ntau,2))*Inp[0]
firing_rate_lwtheo +=np.sum(contribution_theo.copy(),axis=1)



In [21]:
### compute low-rank theoretical
contribution_fr = np.zeros((trials,ntau,2,2))
firing_rate_lwfr = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
            for ir in range(1):
                contribution_fr[ktrial,it,ir,0] = np.squeeze(((mean_leigvec_num_series[ktrial,it,ir,0])*eigvchn_series[ktrial,it,ir].real*Inp[0]*NE+(mean_leigvec_num_series[ktrial,it,ir,1])*eigvchn_series[ktrial,it,ir].real*Inp[0]*NI)/(1-eigvchn_series[ktrial,it,ir].real)*(mean_reigvec_num_series[ktrial,it,ir,0]))
                contribution_fr[ktrial,it,ir,1] = np.squeeze(((mean_leigvec_num_series[ktrial,it,ir,0])*eigvchn_series[ktrial,it,ir].real*Inp[0]*NE+(mean_leigvec_num_series[ktrial,it,ir,1])*eigvchn_series[ktrial,it,ir].real*Inp[0]*NI)/(1-eigvchn_series[ktrial,it,ir].real)*(mean_reigvec_num_series[ktrial,it,ir,1]))
    firing_rate_lwfr[ktrial,:,:] = np.ones((ntau,2))*Inp[0]
    firing_rate_lwfr[ktrial,:,:] +=np.sum(contribution_fr[ktrial,:,:,:].copy(),axis=1)


In [22]:
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
''' fill_between '''
ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
''' plot the mean '''
ax.plot(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
# ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ### start with the excitatory neuron population
ax.fill_between(tau_series,np.nanmean(firing_rateeq_mean[:,:,0],axis=0)-np.nanstd(firing_rateeq_mean[:,:,0],axis=0),np.nanmean(firing_rateeq_mean[:,:,0],axis=0)+np.nanstd(firing_rateeq_mean[:,:,0],axis=0),facecolor='tab:red',alpha=0.6)
### error bar with standard deviation of firing_rate_lwfr the axis 0 
std_lwfr = np.nanstd(firing_rate_lwfr[:,:,0],axis=0)
ax.errorbar(tau_series,np.nanmean(firing_rate_lwfr[:,:,0].real,axis=0),yerr=std_lwfr,fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)
# ax.plot(tau_series, np.nanmean(firing_rate_lwfr[:,:,0].real,axis=0),color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.01,0.05)
ax.set_yticks([0,0.05])
ax.set_ylim(-0.01,0.02)
ax.set_yticks([0,0.02])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)

fig,ax=plt.subplots(figsize=(4,3))
ax.fill_between(tau_series, trialavg_sparseiid_dyns[1]*np.ones((ntau,))-trialstd_sparseiid_dyns[1]*np.ones((ntau,)),trialavg_sparseiid_dyns[1]*np.ones((ntau,))+trialstd_sparseiid_dyns[1]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ### start with the excitatory neuron population
ax.fill_between(tau_series,np.nanmean(firing_rateeq_mean[:,:,1],axis=0)-np.nanstd(firing_rateeq_mean[:,:,1],axis=0),np.nanmean(firing_rateeq_mean[:,:,1],axis=0)+np.nanstd(firing_rateeq_mean[:,:,1],axis=0),facecolor='tab:blue',alpha=0.6)
### error bar with standard deviation of firing_rate_lwfr the axis 0 
std_lwfr = np.nanstd(firing_rate_lwfr[:,:,1],axis=0)
ax.errorbar(tau_series,np.nanmean(firing_rate_lwfr[:,:,1].real,axis=0),yerr=std_lwfr,fmt='x',color='tab:blue',ecolor='tab:blue',alpha=1,ls='',elinewidth=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.01,0.05)
ax.set_yticks([0,0.05])
ax.set_ylim(-0.01,0.02)
ax.set_yticks([0,0.02])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)
# ax.set_title(r'$\nu_{E}$',fontsi

Finale figure

In [24]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns /= Inp[0] ### devided by ... give the response function
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.39135997 0.39177034]


In [25]:
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
# ''' fill_between '''
# ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
''' plot the mean '''
ax.plot(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
std_fr = np.nanstd(firing_rateeq_mean[:,:,0]/Inp[0],axis=0)
ax.errorbar(tau_series,np.nanmean(firing_rateeq_mean[:,:,0],axis=0)/Inp[0],yerr=std_fr,fmt='x',color='gray',ecolor='gray',alpha=1,ls='',elinewidth=1.5)
ax.plot(tau_series, np.nanmean(firing_rate_lwfr[:,:,0].real,axis=0)/Inp[0],color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
# ax.set_ylim(-0.01,0.05)
# ax.set_yticks([0,0.05])
ax.set_ylim(-0.,0.8)
ax.set_yticks([0,0.8])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)

fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
# ''' fill_between '''
# ax.fill_between(tau_series, trialavg_sparseiid_dyns[1]*np.ones((ntau,))-trialstd_sparseiid_dyns[1]*np.ones((ntau,)),trialavg_sparseiid_dyns[1]*np.ones((ntau,))+trialstd_sparseiid_dyns[1]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
''' plot the mean '''
ax.plot(tau_series, trialavg_sparseiid_dyns[1]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
std_fr = np.nanstd(firing_rateeq_mean[:,:,0]/Inp[0],axis=0)
std_fr = np.nanstd(firing_rateeq_mean[:,:,1]/Inp[0],axis=0)
ax.errorbar(tau_series,np.nanmean(firing_rateeq_mean[:,:,1],axis=0)/Inp[0],yerr=std_fr,fmt='x',color='gray',ecolor='gray',alpha=1,ls='',elinewidth=1.5)
ax.plot(tau_series, np.nanmean(firing_rate_lwfr[:,:,1].real,axis=0)/Inp[0],color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.,0.8)
ax.set_yticks([0,0.8])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)

In [77]:
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
# ax.plot(tau_series, firing_rate0_mean[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ### start with the excitatory neuron population
# ax.plot(tau_series, np.nanmean(firing_rateeq_mean[:,:,0],axis=0),marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
alphass=0.95
ax.errorbar(tau_series[:],np.nanmean(firing_rateeq_mean[:,:,0].real,axis=0),yerr=np.nanstd(firing_rateeq_mean[:,:,0].real,axis=0),fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.1)
ax.set_yticks([0,0.1])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)
# ax.set_title(r'$\nu_{E}$',fontsi

In [78]:
### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### contribution from the first rank-1 component
ax.plot(tau_series, contribution_theo[:,0,0],marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.1)
ax.set_yticks([0,0.1])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)